In [55]:
import os
import mlflow
import numpy as np


In [56]:

os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://minio:9000"

In [89]:
import mlflow.sklearn
import requests
import json
from mlflow import MlflowClient

# Set up MLFlow logging
mlflow.set_tracking_uri("http://mlflow_server:5000")
mlflow.set_experiment("my_experiment")
client = MlflowClient()



In [90]:

# Get the best run
runs = client.search_runs(experiment_id, order_by=["metrics.accuracy DESC"], max_results=1)
best_run_id = runs[0].info.run_id
print(best_run_id)


5e17107d41a44524b23e9b3796113e8d


In [91]:
import mlflow
logged_model = f'runs:/{best_run_id}/logistic_model'

# Load model as a PyFuncModel.
loaded_model = mlflow.sklearn.load_model(logged_model)


In [92]:
loaded_model

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', LogisticRegression(random_state=42))])

In [173]:
# Deploy the model to the MLFlow model registry
mlflow.register_model(logged_model, "production_logistic_model")


Registered model 'production_logistic_model' already exists. Creating a new version of this model...
2023/04/16 21:22:29 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: production_logistic_model, version 6
Created version '6' of model 'production_logistic_model'.


<ModelVersion: creation_timestamp=1681680149270, current_stage='None', description='', last_updated_timestamp=1681680149270, name='production_logistic_model', run_id='5e17107d41a44524b23e9b3796113e8d', run_link='', source='s3://mlflow/1/5e17107d41a44524b23e9b3796113e8d/artifacts/logistic_model', status='READY', status_message='', tags={}, user_id='', version='6'>

In [174]:
mlflow.set_tracking_uri("http://mlflow_server:5000")
!mlflow models serve -m "models:/production_logistic_model/5"



Traceback (most recent call last):
  File "/opt/conda/bin/mlflow", line 8, in <module>
    sys.exit(cli())
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1055, in main
    rv = self.invoke(ctx)
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1657, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1657, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1404, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 760, in invoke
    return __callback(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/mlflow/models/cli.py", line 105, in serve
    return get_flavo

In [175]:
from flask import Flask, jsonify, request
import mlflow.sklearn
import pandas as pd

app = Flask(__name__)

@app.route("/predict", methods=["POST"])
def predict():
    # Get the JSON data from the request
    input_data = request.get_json()
    
    # Convert the JSON data to a pandas DataFrame
    input_df = pd.DataFrame.from_dict(input_data, orient="index").T
    
    # Make a prediction using the loaded model
    prediction = loaded_model.predict(input_df)
    
    # Convert the prediction to a JSON response
    response = {"prediction": prediction.tolist()}
    
    return jsonify(response)
